# Knight move on keypad

## Import functions

In [ ]:
import sys, os

# Add the parent directory to the sys.path
if "__file__" in globals().keys():
    sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
else:
    sys.path.append(os.path.dirname(os.getcwd()))

In [ ]:
from src.table_data import get_table_data
from src.positions import get_vowel_positions, get_missing_positions, get_coordinate_bounds
from src.sequence_check import check_inside_bounds, check_in_avoided_coordinate
from src.sequence_create import get_all_sequences
from src.vowel_check import has_at_most_two_vowels


## Initial tests

In [ ]:
# Create the main coordinate data containing the letter/number grid
# This creates coordinate tuples in format (row, col, value) for a 4x5 grid
coordinate_data = get_table_data()
print("Keypad (Coordinate Data):")
print("First few coordinates:")
for i, coord in enumerate(coordinate_data[:10]):
    print(f"  {coord}")
print(f"  ... and {len(coordinate_data) - 10} more coordinates")
print()

# Find all missing/empty positions in the coordinate data
# These positions will be avoided during sequence generation
miss_pos = get_missing_positions(coordinate_data)
print(f"Missing positions (to avoid): {miss_pos}")

# Find all vowel positions (A, E, I, O, U) in the coordinate data  
# These are special positions that we want to limit visits to
vo_pos = get_vowel_positions(coordinate_data)
print(f"Vowel positions (limited visits): {vo_pos}")

# Extract the numeric bounds of the coordinate data
# This gives us [x_min, x_max, y_min, y_max] for boundary checking
bounds_dict = get_coordinate_bounds(coordinate_data)
idx_bounds = [bounds_dict['x_min'], bounds_dict['x_max'], bounds_dict['y_min'], bounds_dict['y_max']]
print(f"Index bounds [x_min, x_max, y_min, y_max]: {idx_bounds}")
print()

In [ ]:
# Create a filter function that validates knight move sequences
# A valid sequence must:
# 1. Stay within the coordinate boundaries (check_inside_bounds)
# 2. Avoid all missing/empty positions (not check_in_avoided_coordinate)
filter = lambda sequence: (check_inside_bounds(sequence, *idx_bounds)) and (not check_in_avoided_coordinate(sequence, miss_pos))


In [ ]:
# Test sequences
sequences = get_all_sequences(1, 1, 3, location_list=vo_pos, max_hit=2,
               filter=filter,
               )

print(f"Total number of sequences: {len(sequences)}")
print("Sequences:")

# Create a lookup dictionary for coordinate values
coord_lookup = {(x, y): value for x, y, value in coordinate_data}

for sequence in sequences:
    sequence_string = ("".join([coord_lookup[i, j] for i, j in sequence]))
    print(sequence)
    print(sequence_string)


In [ ]:
# Check the total number of sequences

x_i = 1
y_i = 1
step_i = 3

print("Create all sequences then apply filter:")

all_sequence_list = get_all_sequences(x_i, y_i, step_i, location_list=vo_pos, max_hit=2)

print(f"Total number of sequences inside and outside the bounds: {len([sequence for sequence in all_sequence_list if not check_inside_bounds(sequence, *idx_bounds)]) + len([sequence for sequence in all_sequence_list if check_inside_bounds(sequence, *idx_bounds)])}")

print(f"Total number of sequences hitting or not hitting the missing positions: {len([sequence for sequence in all_sequence_list if not check_in_avoided_coordinate(sequence, miss_pos)]) + len([sequence for sequence in all_sequence_list if check_in_avoided_coordinate(sequence, miss_pos)])}")

sequence_list_after_filter = [sequence for sequence in all_sequence_list if filter(sequence)]

print(f"Total number of sequences after applying the filter: {len(sequence_list_after_filter)}")

print("Create all sequences with filter:")

all_sequence_list = get_all_sequences(x_i, y_i, step_i, location_list=vo_pos, max_hit=2, filter=filter)

print(f"Total number of sequences inside and outside the bounds: {len([sequence for sequence in all_sequence_list if not check_inside_bounds(sequence, *idx_bounds)]) + len([sequence for sequence in all_sequence_list if check_inside_bounds(sequence, *idx_bounds)])}")

print(f"Total number of sequences hitting or not hitting the missing positions: {len([sequence for sequence in all_sequence_list if not check_in_avoided_coordinate(sequence, miss_pos)]) + len([sequence for sequence in all_sequence_list if check_in_avoided_coordinate(sequence, miss_pos)])}")

sequence_list_with_filter = [sequence for sequence in all_sequence_list if filter(sequence)]

print(f"Total number of sequences: {len(sequence_list_with_filter)}")

# Create a lookup dictionary for coordinate values
coord_lookup = {(x, y): value for x, y, value in coordinate_data}

# Transform each sequence to sequence_string
print("\nSequence strings from sequence_list_with_filter:")
for i, sequence in enumerate(sequence_list_with_filter):
    sequence_string = "".join([coord_lookup[i, j] for i, j in sequence])
    print(f"Sequence {i+1}: {sequence} -> '{sequence_string}'")


## Efficiency analysis

In [ ]:
import time
from datetime import datetime

def time_execution(func, *args, description="Function execution", **kwargs):
    """
    Time the execution of a function and return results with timing information.
    
    Parameters:
    -----------
    func : callable
        The function to time
    *args : tuple
        Positional arguments for the function
    description : str
        Description of what's being timed
    **kwargs : dict
        Keyword arguments for the function
    
    Returns:
    --------
    tuple
        (result, execution_time_seconds)
    """
    print(f"\n🕐 Starting: {description}")
    print(f"📅 Start time: {datetime.now().strftime('%H:%M:%S.%f')[:-3]}")
    
    start_time = time.perf_counter()
    result = func(*args, **kwargs)
    end_time = time.perf_counter()
    
    execution_time = end_time - start_time
    
    print(f"⏱️  Execution time: {execution_time:.4f} seconds")
    print(f"📅 End time: {datetime.now().strftime('%H:%M:%S.%f')[:-3]}")
    
    if execution_time < 0.001:
        print(f"⚡ Very fast: {execution_time*1000:.2f} milliseconds")
    elif execution_time < 1:
        print(f"🏃 Fast: {execution_time*1000:.1f} milliseconds") 
    elif execution_time < 10:
        print(f"🚶 Moderate: {execution_time:.2f} seconds")
    else:
        print(f"🐌 Slow: {execution_time:.1f} seconds")
    
    return result, execution_time

# Time the sequence generation for different steps
print("="*60)
print("TIMING ANALYSIS FOR SEQUENCE GENERATION")
print("="*60)

step_i = 3
x_i, y_i = 1, 1
max_hit = 2

# Time different operations
timing_results = {}

# 1. Time basic sequence generation
sequences, time1 = time_execution(
    get_all_sequences, x_i, y_i, step_i,
    description=f"Basic sequence generation (step {step_i})"
)
timing_results['basic_sequences'] = (len(sequences), time1)

# 2. Time sequence generation with vowel location constraints
sequences_vowel, time2 = time_execution(
    get_all_sequences, x_i, y_i, step_i,
    location_list=vo_pos, max_hit=max_hit,
    description=f"Sequence generation with vowel constraints (step {step_i})"
)
timing_results['vowel_constrained'] = (len(sequences_vowel), time2)

# 3. Time sequence generation with filter
sequences_filtered, time3 = time_execution(
    get_all_sequences, x_i, y_i, step_i,
    location_list=vo_pos, max_hit=max_hit, filter=filter,
    description=f"Sequence generation with filter (step {step_i})"
)
timing_results['filtered'] = (len(sequences_filtered), time3)

# 4. Time the vowel checking loop
def vowel_check_loop():
    total_sequences = 0
    # Create a lookup dictionary for coordinate values
    coord_lookup = {(x, y): value for x, y, value in coordinate_data}
    
    for x_i in range(1, 5):
        for y_i in range(1, 6):
            all_sequence_list = get_all_sequences(x_i, y_i, step_i, location_list=vo_pos, max_hit=max_hit, filter=filter)
            for sequence in all_sequence_list:
                sequence_string = "".join([coord_lookup[i, j] for i, j in sequence])
                if has_at_most_two_vowels(sequence_string):
                    total_sequences += 1
    return total_sequences

vowel_count, time4 = time_execution(
    vowel_check_loop,
    description=f"Complete vowel analysis loop (step {step_i})"
)
timing_results['vowel_analysis'] = (vowel_count, time4)

# Display summary
print("\n" + "="*60)
print("TIMING SUMMARY")
print("="*60)
print(f"{'Operation':<35} {'Sequences':<10} {'Time (s)':<12} {'Sequences/sec':<12}")
print("-" * 60)

for operation, (seq_count, exec_time) in timing_results.items():
    rate = seq_count / exec_time if exec_time > 0 else 0
    print(f"{operation:<35} {seq_count:<10} {exec_time:<12.4f} {rate:<12.1f}")

print("\n📊 Performance Insights:")
print(f"• Filter reduces sequences by {((timing_results['basic_sequences'][0] - timing_results['filtered'][0]) / timing_results['basic_sequences'][0] * 100):.1f}%")
print(f"• Vowel constraints reduce sequences by {((timing_results['basic_sequences'][0] - timing_results['vowel_constrained'][0]) / timing_results['basic_sequences'][0] * 100):.1f}%")
print(f"• Processing rate: {timing_results['vowel_analysis'][0] / timing_results['vowel_analysis'][1]:.1f} vowel checks per second")


## Test different methods to get the sequences

In [ ]:
import time

x_min, x_max = bounds_dict['x_min'], bounds_dict['x_max']
y_min, y_max = bounds_dict['y_min'], bounds_dict['y_max']
max_hit = 2

### Apply filter when creating the sequences

In [ ]:
# Iterate through different sequence lengths and total_sequences with timing

print("Sequence Analysis with Timing:")
print("=" * 60)
print(f"{'Step':<6} {'Sequence Length':<12} {'Total Sequences':<12} {'Time (s)':<10} {'Seq/sec':<10}")
print("-" * 60)

for step_i in [3, 5, 7, 9]:
    start_time = time.perf_counter()
    
    total_sequences = 0
    for x_i in range(x_min, x_max + 1):
        for y_i in range(y_min, y_max + 1):
            all_sequence_list = get_all_sequences(x_i, y_i, step_i, location_list=vo_pos, max_hit=max_hit, filter=filter)
            total_sequences += len(all_sequence_list)
    
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    sequence_length = step_i + 1  # step_i is number of moves, sequence length is moves + 1
    seq_per_sec = total_sequences / execution_time if execution_time > 0 else 0
   
    print(f"{step_i:<6} {sequence_length:<12} {total_sequences:<12} {execution_time:<10.4f} {seq_per_sec:<10.1f}")

print("=" * 60)


### Apply filter after creating the sequences

In [ ]:
# Iterate through different sequence lengths with vowel checking and timing

print("Sequence Analysis with Vowel Checking and Timing:")
print("=" * 70)
print(f"{'Step':<6} {'Sequence Length':<12} {'Total Sequences':<12} {'Time (s)':<10} {'Seq/sec':<10}")
print("-" * 70)

# Create a lookup dictionary for coordinate values (once, outside the loop)
coord_lookup = {(x, y): value for x, y, value in coordinate_data}

for step_i in [3, 5, 7, 9]:
    start_time = time.perf_counter()
    
    total_sequences = 0
    for x_i in range(x_min, x_max + 1):
        for y_i in range(y_min, y_max + 1):

            all_sequence_list = get_all_sequences(x_i, y_i, step_i)
            sequence_list_final = [sequence for sequence in all_sequence_list if filter(sequence)]

            for sequence in sequence_list_final:
                sequence_string = ("".join([coord_lookup[i, j] for i, j in sequence]))
                
                if has_at_most_two_vowels(sequence_string):
                    total_sequences += 1

    end_time = time.perf_counter()
    execution_time = end_time - start_time
    sequence_length = step_i + 1  # step_i is number of moves, sequence length is moves + 1
    seq_per_sec = total_sequences / execution_time if execution_time > 0 else 0
    
    print(f"{step_i:<6} {sequence_length:<12} {total_sequences:<12} {execution_time:<10.4f} {seq_per_sec:<10.1f}")

print("=" * 70)
